<p> The program is the following work of Model 1, Here I present a more complex network</p>
<p> All data about pipes are estimated and assigned based on a rough map</p>

In [1]:
import sys
sys.path.append("..")    
from allthethings import PyNetwork, PyPipe_ps
from allthethings import PyBC_opt_dh
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
from writeit import *
import pickle
import time
from __future__ import division 
import pickle

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Build Model
fi = "../indata/Model2/Model2_Epipe-2.0.inp"    
fc = "../indata/Model2/Model2_Epipe-2.0.config" 
mtype=1
n0=PyNetwork(fi,fc,mtype)

In [3]:
# no longer to create .inp file and .config file

l1=100
mtype =1
from time import sleep
for ratio in [2]:
    l2 = int(l1*ratio)
    Ls = np.array([l1]*5+[l2]*4)  #no comma between numbers
    print Ls/2
    fi = "../indata/Model2/Model2_Epipe_l_%d_%d.inp"%(l1,l2)
    fc = "../indata/Model2/Model2_Epipe_l_%d_%d.config"%(l1,l2)
    n1=PyNetwork(fi,fc,mtype)

    v = 2
    Q_in1 = list(n1.Ds[0]**2*pi/4*v*np.ones(int(n1.M*0.7)))
    Q_in2 = list(np.zeros(int(n1.M*0.3)+1))
    Q_in = Q_in1+Q_in2
    jt = n1.nodeTypes
    dt = n1.T/n1.M  
    print dt
    aa=arange(0,len(jt)) 
    for j in range(len(aa[n1.nodeTypes==1])):#find the junction1s and set boundary value time series to .1*(pipe diameter)
        bvt = n1.Ds[5]*np.ones(n1.M+1)
        n1.setbVal(j,bvt) 
    n1.setbVal(0,Q_in)
    n1.setbVal(1,n1.Ds[4]*np.ones(n1.M+1))

    Np = n1.Nedges # number of pipes
    p1 = PyPipe_ps(n1.Ns[0], n1.Ds[0],n1.Ls[0], n1.M, n1.a[0])      
    h0 = p1.AofH(10,False)
    A00 = h0*np.ones(n1.Ns[0])  
    Q00 = Q_in[0]*np.ones(n1.Ns[0])
    h1 = p1.AofH(0.01,False)
    n1.setIC(0,A00,Q00) 
    for k in range(2,Np):                     #pipe number starts from 0, ends at Np-1. Here from 2 to Np-1
        A00 = h1*np.ones(n1.Ns[k])  #set other pipes to 1e-3*(pipe diameter) to avoid near-zero roundoff issues
        Q00 = np.zeros(n1.Ns[k])
        n1.setIC(k, A00, Q00) 
    
    V0 = n1.getTotalVolume()                 
    n1.runForwardProblem(dt) 
    V1= n1.getTotalVolume()
    print "The total run time is %f s, the total volume change is %f %f cubic meters" %(n1.solve_time, V1,V0)

    # 0.  Create Data

    # H [time step][pipe][grid] shows pressure head at a certain time step along a certain pipe with certain grid
    t_store_interval = 1
    M_store_interval = int(t_store_interval/dt)
    H = [] 
    for time in xrange(0,n1.M+1,M_store_interval):
        H_t = [] 
        for pipe in xrange(0,9):
            H_t_pipe = n1.pressureSpaceSeries(pipe,time)
            H_t.append(H_t_pipe)
        H.append(H_t)
    print len(H)
    # x [pipe][grid] create x axis for a certain pipe using grids
    x_interest=[]
    for pipe in xrange(0,9): 
        x = np.linspace(0,n1.Ls[pipe],n1.Ns[pipe])
        x_interest.append(x)
    Data={"H":H,"x":x_interest}

    file=open("/home/xin/pipes/examples/output_data/EpipeFig/length_ratio=%.2f"%(1/ratio),"wb")
    pickle.dump(Data,file)
    file.close()
    n1.reset()
    sleep(30)

[  50.   50.   50.   50.   50.  100.  100.  100.  100.]
0.02
The total run time is 248.124468 s, the total volume change is 14.066513 7.949091 cubic meters
2401


In [ ]:
Ds=n0.Ds
Mrs=n0.Mrs
jt=n0.nodeTypes     # number of pipes connecting to a certain junction
# all pipes are opening except the inflow pipe
bv=[0]*len(jt)
bt=[1]
r=[0]
for i in xrange(1,len(jt)):
    if jt[i]==1:
        bt.append(2)
        r.append(0)
    else:
        bt.append(0)
        r.append(1)
Np=n0.Nedges        #number of pipes
h0s=[0.001]*Np
q0s=[0.000]*Np      # carefully set it, it can determine if the model can be run 
T=2400
a=90
#M = int(T*a/(mdx*.8))
Mi= 10000
M = 200000
#M= (M+Mi-M%Mi)  
elevs=[1,1,1,1,1,1,0,0,0,0]



In [ ]:
def rewritePipes(fn, oldinp, Ns, Ls, Mrs, Ds, jt, bt, bv, r, h0s, q0s, T, M, Mi, a,elevs):
    newconfig = fn + (".config")
    newinp = fn + ".inp"
#    Mi = 10
# config file section titles
    Ptitle = "[PIPE_INFO]\n\
;-------------------------------------------\n\
;initial  initial\n; ID    N       h       Q\n\
;-------------------------------------------\n"
    Jtitle = "[JUNCTION_INFO]\
;-------------------------------------------------------------------------------------\n\
;{-----for junction1s-----} | {--for junction2s--} | {------for junction3s-------}|\n\
;ID     jtype   bvaltype  bval   reflect   | offset   valveopen   | offset01   offset02   offset12 |\n\
;------------------------------------------------------------------------------------\n"
    Ttitle = "[TIME_INFO]\
;---------------------------------------\n\
;T (s)           M        Mi     a  (m/s)\n\
;-----------------------------------------\n"
    
# open config file and write
    with open(newconfig, 'w') as fw:
        fw.write(Ptitle)
        for j in range(len(Ns)):
            fw.write("%d    %d   %2.5f   %2.3f\n" %
                     (j, int(Ns[j]), h0s[j], q0s[j]))
        fw.write("\n")
        fw.write(Jtitle)
        for k in range(len(jt)):
            fw.write("%d     %d     %d     %d     %d     %d     %d     %d     %d     %d\n" % (
                k, jt[k], bt[k], bv[k], r[k], 0, 1, 0, 0, 0))
        fw.write("\n")
        fw.write(Ttitle)
        fw.write("%3.3f       %d         %d        %.1f" % (T, M, Mi, a))
    fw.close()
# now open old inp file and change pipe properties, leaving everthing else
# the same
    count = 0
    count1 = 0
    count2 = 0
    with open(oldinp, 'r') as fold:
        with open(newinp, 'wb') as fnew:
            for line in fold:
                s = line.split()
                if '[' in line:
                    fnew.write(line)
                    count += 1
                elif (count ==2) and len(s) >1 and (';' not in s[0]): 
                    fnew.write("%d %15s %2.3f %15s0 %30s ;\n"%(count1," ",elevs[count1]," "," "))
                    count1 +=1
                    #print count1
                    if count1>=len(jt):
                        count+=1   
                elif (count < 6) and len(s) > 1 and (';' not in s[0]):
                    #print 'yes'
                    fnew.write("%s %23s %s %23s %s %17s %4.1d %10s %2.2f %10s %1.4f %10s 0 %12s open\n" % \
                                (s[0]," ",s[1]," ", s[2]," ", Ls[count2]," ", Ds[count2]," ", Mrs[count2],"",""))
                    count2 += 1
                    if count2 >= len(Ns):
                        count += 1
                else:
                    fnew.write(line)
    #print "new files are %s and %s" % (newinp, newconfig)
    return (newinp, newconfig)

In [ ]:
l1=100
for ratio in [2]:
    if ratio == 2:  # The error message requires to increase dt
        M = 120000
    l2 = int(l1*ratio)
    Ls = np.array([l1]*5+[l2]*4)  #no comma between numbers
    print Ls/2
    Ns = [int(l) for l in Ls/2] 
    mdx = min([Ls[i]/Ns[i] for i in range (Np)])
    fn = "../indata/Model2/Model2_Epipe_l_%d_%d"%(l1,l2)           # new file name for .inp and .config
    oldinp = "../indata/Model2/Model2_Epipe-2.0.inp"                   # old inp file
    (fi2,fc2) = rewritePipes(fn,oldinp, Ns, Ls, Mrs, Ds, jt, bt, bv, r, h0s, q0s, T, M, Mi, a, elevs)
#fi = "../indata/Model2/Model2_Epipe_r_%.2f_%.2f.inp"%(r1,r2)
#fc = "../indata/Model2/Model2_Epipe_r_%.2f_%.2f.config"%(r1,r2)
    n1=PyNetwork(fi2,fc2,mtype)
    v = 2
    Q_in1 = list(n1.Ds[0]**2*pi/4*v*np.ones(int(n1.M*0.7)))
    Q_in2 = list(np.zeros(int(n1.M*0.3)+1))
    Q_in = Q_in1+Q_in2
    jt = n1.nodeTypes
    dt = n1.T/n1.M  
    print dt
    aa=arange(0,len(jt)) 
    for j in range(len(aa[n1.nodeTypes==1])):#find the junction1s and set boundary value time series to .1*(pipe diameter)
        bvt = n1.Ds[5]*np.ones(n1.M+1)
        n1.setbVal(j,bvt) 
    n1.setbVal(0,Q_in)
    n1.setbVal(1,n1.Ds[4]*np.ones(n1.M+1))

    V0 = n1.getTotalVolume()                 
    n1.runForwardProblem(dt) 
    V1= n1.getTotalVolume()
    print "The total run time is %f s, the total volume change is %f %f cubic meters" %(n1.solve_time, V1,V0)

    # 0.  Create Data

    # H [time step][pipe][grid] shows pressure head at a certain time step along a certain pipe with certain grid
    t_store_interval = 1
    M_store_interval = int(t_store_interval/dt)
    H = [] 
    for time in xrange(0,n1.M+1,M_store_interval):
        H_t = [] 
        for pipe in xrange(0,9):
            H_t_pipe = n1.pressureSpaceSeries(pipe,time)
            H_t.append(H_t_pipe)
        H.append(H_t)
    print len(H)
    # x [pipe][grid] create x axis for a certain pipe using grids
    x_interest=[]
    for pipe in xrange(0,9): 
        x = np.linspace(0,n1.Ls[pipe],n1.Ns[pipe])
        x_interest.append(x)
    Data={"H":H,"x":x_interest}

    file=open("/home/xin/pipes/examples/output_data/EpipeFig/length_ratio=%.2f"%(1/ratio),"wb")
    pickle.dump(Data,file)
    file.close()
    n1.reset()

In [1]:
from time import sleep
sleep(4)
